In [1]:
from utils import *
import importlib
importlib.reload(sys.modules['utils'])


<module 'utils' from '/Users/jzhen3/Desktop/Mini_DIVA/utils.py'>

In [2]:
data, X, y, Xinds, yinds = read_dataset(dataset="fradulent_transactions")#input the dataset
impute_fraction = 0.2
random_state = 42
# thres = int(len(X)*0.75)
num_vars, cat_vars = get_num_cat_vars(X)


In [3]:
data

,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD
0,623.71,9272650,107,0
1,1478.04,14202612,164,1
2,510.91,15071963,174,0
3,426.10,5065768,58,0
4,1622.20,13071312,151,1
...,...,...,...,...
9104,1178.53,5995564,69,1
9105,1150.08,209871,2,1
9106,171.84,13697079,158,0
9107,408.33,2990685,34,0


In [5]:
X.head()

,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS
0,-0.188244,0.307583,0.310951
1,0.075047,-0.388991,-0.390081
2,-0.257476,1.579298,1.580388
3,-0.309530,-0.614933,-0.617443
4,0.233183,-0.206385,-0.200613


SELECT the best params for KNN model

In [4]:
impute_fraction = 0.2
random_state = 42
impute_method = 'KNNImputer'
error_method = "mix"
n_neighbors = 7

thres = int(len(X)*0.75)
err_dic = []

for w in ['uniform', 'distance']:
    for k in range(20,40):
        X_miss, X_miss_idx = set_fraction_missing(
            X, fraction=impute_fraction, random_state=random_state
        )
        X_imp, X_le = impute(
            X_miss,
            X,
            y,
            encoding="one-hot",
            train_idx=X.index[:thres],
            test_idx=X.index[thres:],
            method=impute_method,
            k_num=k,
            weight=w
        )
        error = evaluate_imputation(
            X,
            X_imp.iloc[thres:],
            X_le.iloc[thres:],
            X_miss_idx.iloc[thres:],
            error_method=error_method,
        )
        err_dic.append((w, k, error))

In [5]:
best_weight = min(err_dic, key=lambda x: x[-1])[0]
best_key = min(err_dic, key=lambda x: x[-1])[1]
best_value = min(err_dic, key=lambda x: x[-1])[2]
print((f'best_weight: {best_weight}, best_key: {best_key}, best_value: {best_value}'))

best_weight: distance, best_key: 39, best_value: 0.8380399199947576


SELECT the best CustoMICE model

In [6]:
impute_fraction = 0.2
random_state = 42
impute_method = 'customMICE'
error_method = "mix"

iteration = range(1, 4)
methodN = ['linear','lasso','ridge']
methodC = ['logistic', 'random_forest']
solver = ['sags', 'saga', 'newton-cg', 'liblinear', 'lbfgs']


thres = int(len(X)*0.75)
err_dic2 = []

for iter in iteration:
    for n in methodN:
        for c in methodC:
            if c == "logistic":
                for s in solver:
                    X_miss, X_miss_idx = set_fraction_missing(
                    X, fraction=impute_fraction, random_state=random_state)
                    X_imp, X_le = impute(
                        X_miss,
                        X,
                        y,
                        encoding="one-hot",
                        train_idx=X.index[:thres],
                        test_idx=X.index[thres:],
                        method=impute_method,
                        k_num=k,
                        weight=w,
                        iteration = iter,
                        methodN = n,
                        methodC = c,
                        solver = s,
                    )
                    error = evaluate_imputation(
                        X,
                        X_imp.iloc[thres:],
                        X_le.iloc[thres:],
                        X_miss_idx.iloc[thres:],
                        error_method=error_method,
                    )
                    err_dic2.append((iter, n, c, s, error))
            else:
                X_miss, X_miss_idx = set_fraction_missing(
                    X, fraction=impute_fraction, random_state=random_state)
                X_imp, X_le = impute(
                    X_miss,
                    X,
                    y,
                    encoding="one-hot",
                    train_idx=X.index[:thres],
                    test_idx=X.index[thres:],
                    method=impute_method,
                    k_num=k,
                    weight=w,
                    iteration = iter,
                    methodN = n,
                    methodC = c,
                )
                error = evaluate_imputation(
                    X,
                    X_imp.iloc[thres:],
                    X_le.iloc[thres:],
                    X_miss_idx.iloc[thres:],
                    error_method=error_method,
                )
                err_dic2.append((iter, n, c, '', error))

In [7]:
err_dic2

[(1, 'linear', 'logistic', 'sags', 0.6381526684293827),
 (1, 'linear', 'logistic', 'saga', 0.6381526684293827),
 (1, 'linear', 'logistic', 'newton-cg', 0.6381526684293827),
 (1, 'linear', 'logistic', 'liblinear', 0.6381526684293827),
 (1, 'linear', 'logistic', 'lbfgs', 0.6381526684293827),
 (1, 'linear', 'random_forest', '', 0.6381526684293827),
 (1, 'lasso', 'logistic', 'sags', 0.6381526380797442),
 (1, 'lasso', 'logistic', 'saga', 0.6381526380797442),
 (1, 'lasso', 'logistic', 'newton-cg', 0.6381526380797442),
 (1, 'lasso', 'logistic', 'liblinear', 0.6381526380797442),
 (1, 'lasso', 'logistic', 'lbfgs', 0.6381526380797442),
 (1, 'lasso', 'random_forest', '', 0.6381526380797442),
 (1, 'ridge', 'logistic', 'sags', 0.6381526684189271),
 (1, 'ridge', 'logistic', 'saga', 0.6381526684189271),
 (1, 'ridge', 'logistic', 'newton-cg', 0.6381526684189271),
 (1, 'ridge', 'logistic', 'liblinear', 0.6381526684189271),
 (1, 'ridge', 'logistic', 'lbfgs', 0.6381526684189271),
 (1, 'ridge', 'random_fo

In [8]:
best_iter = min(err_dic2, key=lambda x: x[-1])[0]
best_num_model = min(err_dic2, key=lambda x: x[-1])[1]
best_cat_model = min(err_dic2, key=lambda x: x[-1])[2]
best_solver = min(err_dic2, key=lambda x: x[-1])[3]
best_val = min(err_dic2, key=lambda x: x[-1])[4]

print((best_iter, best_num_model, best_cat_model, best_solver,best_val))

(3, 'lasso', 'logistic', 'sags', 0.6381249381833276)


In [9]:
print((f'best_iter: {best_iter}, best_num_model: {best_num_model}, best_cat_model: {best_cat_model}, best_val: {best_val}'))

best_iter: 3, best_num_model: lasso, best_cat_model: logistic, best_val: 0.6381249381833276


In [10]:
impute_fraction = 0.2
random_state = 42
impute_method = 'MissForest'
error_method = "mix"
n_neighbors = 7

thres = int(len(X)*0.75)
err_dic3 = []
ini=['median','mean']
max_iters=range(1,6)
for i in ini:
    for m in max_iters:
        X_miss, X_miss_idx = set_fraction_missing(
            X, fraction=impute_fraction, random_state=random_state
        )
        X_imp, X_le = impute(
            X_miss,
            X,
            y,
            encoding="one-hot",
            train_idx=X.index[:thres],
            test_idx=X.index[thres:],
            method=impute_method,
            initial_guess = i,
            max_iter=m,
        )
        error = evaluate_imputation(
            X,
            X_imp.iloc[thres:],
            X_le.iloc[thres:],
            X_miss_idx.iloc[thres:],
            error_method=error_method,
        )
        err_dic3.append((i, m, error))

In [11]:
best_guess = min(err_dic3, key=lambda x: x[-1])[0]
best_max_iter = min(err_dic3, key=lambda x: x[-1])[1]
best_vals = min(err_dic3, key=lambda x: x[-1])[2]

print((best_guess, best_max_iter,best_vals))

('mean', 1, 0.6411547455448845)


In [12]:
print((f'best_guess: {best_guess}, best_max_iter: {best_max_iter}, best_vals: {best_vals}'))

best_guess: mean, best_max_iter: 1, best_vals: 0.6411547455448845
